<a href="https://colab.research.google.com/github/laneboi/playground/blob/master/Cython_for_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# install dependencies
!pip install cython
!pip install spacy

In [0]:
# create rectangles

from random import random


class Rectangle(object):
    def __init__(self, w, h):
        self.w = w
        self.h = h
        
    def area(self):
        return self.w * self.h
    
    
def check_rectangles_py(rectangles, threshold):
    n_out = 0
    for rectangle in rectangles:
        if rectangle.area() > threshold:
            n_out += 1
    return n_out


def main_rectangles_slow():
    n_rectangles = int(10e6)
    rectangles = [Rectangle(random(), random()) for _ in range(n_rectangles)]
    n_out = check_rectangles_py(rectangles, threshold=0.25)
    print(n_out)

In [0]:
# time it

%%time
main_rectangles_slow()

4033290
CPU times: user 13.6 s, sys: 1.27 s, total: 14.9 s
Wall time: 14.9 s


In [0]:
# load cython

%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [0]:
# faster cython implementation

%%cython
from cymem.cymem cimport Pool
from random import random


cdef struct Rectangle:
    float w
    float h
    

cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    cdef int n_out = 0
    
    for rectangle in rectangles[:n_rectangles]:
        if rectangle.w * rectangle.h > threshold:
            n_out += 1
    return n_out


def main_rectangles_cy():
    cdef int n_rectangles = 1000000
    cdef float threshold = 0.25
    
    cdef Pool mem = Pool()
    cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    
    for i in range(n_rectangles):
        rectangles[i].w = random()
        rectangles[i].h = random()
    
    n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
    print(n_out)

In [0]:
%%time
main_rectangles_cy()

403063
CPU times: user 49 ms, sys: 992 µs, total: 50 ms
Wall time: 49.7 ms


In [0]:
# install spacy
import spacy.cli


spacy.cli.download('en')
nlp = spacy.load('en')

multi_sentence = 'Sipping on straight chlorine, let the vibe slide over me. This beat is a chemical. Beat is a chemical.' \
                 'When I leave, don\'t save my seed. I\'ll be back when it\'s all complete. The moment is magical. Moment is magical.' \
                 'Fall out of formation. Venom on my tongue, depending this time. Poisonous vibration helps my body run.' \
                 'I\'m running for my life, running for my life.'

parsed_data = nlp(multi_sentence)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
# POS tagging

for span in parsed_data.sents:
    sent = [parsed_data[i] for i in range(span.start, span.end)]
    break
    
for token in sent:
    print(token.orth_, token.pos_)

Sipping VERB
on ADP
straight ADJ
chlorine NOUN
, PUNCT
let VERB
the DET
vibe NOUN
slide VERB
over ADP
me PRON
. PUNCT


In [0]:
# dependency tagging

example = "there's an infestation in my mind's imaginagion, " \
          "I hope they choke, 'cause I'm smoking them out the basement"

parsed_ex = nlp(example)

for token in parsed_ex:
    print(f'{token.orth_}\t\t{token.dep_}\t\t{token.head.orth_}')

there		expl		's
's		ccomp		hope
an		det		infestation
infestation		attr		's
in		prep		infestation
my		poss		mind
mind		poss		imaginagion
's		case		mind
imaginagion		pobj		in
,		punct		hope
I		nsubj		hope
hope		ROOT		hope
they		nsubj		choke
choke		ccomp		hope
,		punct		hope
'cause		mark		smoking
I		nsubj		smoking
'm		aux		smoking
smoking		advcl		hope
them		dobj		smoking
out		prt		smoking
the		det		basement
basement		dobj		smoking
